In [1]:
!java -version

java version "22.0.2" 2024-07-16
Java(TM) SE Runtime Environment (build 22.0.2+9-70)
Java HotSpot(TM) 64-Bit Server VM (build 22.0.2+9-70, mixed mode, sharing)


In [2]:
import os
import re
import tabula
import pandas as pd
import numpy as np

In [3]:
# Set the JAVA_HOME environment variable to the Java installation directory
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk/libexec/openjdk.jdk"

In [4]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [5]:
%pip install -q tabula-py
%pip install selenium requests 

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import time
import os
import re

# Setup Chrome options
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run headless Chrome (no GUI)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1920x1080')

driver = webdriver.Chrome(options=options)

try:
    driver.get("https://www.celcor.com.ph/rate-archives")

    # Click "Show more" until all items are loaded
    while True:
        try:
            # Scroll incrementally to ensure button visibility
            scroll_pause_time = 1
            last_height = driver.execute_script("return document.body.scrollHeight")
            
            while True:
                # Scroll down to bottom
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                # Wait to load page
                time.sleep(scroll_pause_time)

                # Calculate new scroll height and compare with last scroll height
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            show_more_button = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, "//button[@class='btn btn-download-pdf' and @style='visibility: visible;']"))
            )
            
            # Scroll to the button
            driver.execute_script("arguments[0].scrollIntoView(true);", show_more_button)
            time.sleep(1)  # Allow time for the scroll
            
            show_more_button.click()
            time.sleep(2)  # Wait for content to load
        except Exception as e:
            print(f"No more 'Show more' buttons or error: {e}")
            break  # Break the loop if no more "Show more" button is found

    rows = driver.find_elements(By.XPATH, "//div[contains(text(), 'Generation')]")

    # Prepare the download directory
    os.makedirs('pdf_downloads', exist_ok=True)

    for row in rows:
        try:
            # Find the date in the same row
            date_element = WebDriverWait(row, 10).until(
                EC.presence_of_element_located((By.XPATH, ".//div[contains(@class, 'col-md-3 col-lg-2 col-xl-2 col-sm-12 col-12')]"))
            )
            date_text = date_element.text.strip()

            # Convert date text to "mm-yy" format
            match = re.search(r'(\w+)\s+(\d{4})', date_text)
            if match:
                month_str, year_str = match.groups()
                month = {
                    'January': '01', 'February': '02', 'March': '03', 'April': '04',
                    'May': '05', 'June': '06', 'July': '07', 'August': '08',
                    'September': '09', 'October': '10', 'November': '11', 'December': '12'
                }.get(month_str)
                if not month:
                    print(f"Month '{month_str}' not recognized.")
                    continue
                date_formatted = f"{month}-{year_str[-2:]}"

                # Break the loop if the date is May 2022
                if month_str == 'May' and year_str == '2022':
                    print("Reached May 2022. Stopping downloads.")
                    break
            else:
                print(f"Date format not recognized for text: {date_text}")
                continue  # Skip this row if date format is not recognized

            # Find the PDF link
            pdf_link = WebDriverWait(row, 10).until(
                EC.presence_of_element_located((By.XPATH, ".//a[contains(@class, 'download-pdf-link') and contains(@href, '.pdf')]"))
            )
            url = pdf_link.get_attribute('href')

            # Download the PDF
            response = requests.get(url)
            if response.status_code == 200:
                pdf_name = f"gc_table_{date_formatted}.pdf"
                with open(f'pdf_downloads/{pdf_name}', 'wb') as f:
                    f.write(response.content)
                print(f"Downloaded {pdf_name}")
            else:
                print(f"Failed to download PDF from {url}, status code: {response.status_code}")

        except Exception as e:
            print(f"Error processing row: {e}")

finally:
    driver.quit()


No more 'Show more' buttons or error: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7A3A99632+30946]
	(No symbol) [0x00007FF7A3A4E3C9]
	(No symbol) [0x00007FF7A3946FDA]
	(No symbol) [0x00007FF7A399822C]
	(No symbol) [0x00007FF7A399850C]
	(No symbol) [0x00007FF7A39DDCB7]
	(No symbol) [0x00007FF7A39BCAAF]
	(No symbol) [0x00007FF7A39DB041]
	(No symbol) [0x00007FF7A39BC813]
	(No symbol) [0x00007FF7A398A6E5]
	(No symbol) [0x00007FF7A398B021]
	GetHandleVerifier [0x00007FF7A3BCF83D+1301229]
	GetHandleVerifier [0x00007FF7A3BDBDB7+1351783]
	GetHandleVerifier [0x00007FF7A3BD2A03+1313971]
	GetHandleVerifier [0x00007FF7A3ACDD06+245686]
	(No symbol) [0x00007FF7A3A5758F]
	(No symbol) [0x00007FF7A3A53804]
	(No symbol) [0x00007FF7A3A53992]
	(No symbol) [0x00007FF7A3A4A3EF]
	BaseThreadInitThunk [0x00007FF9C940DBE7+23]
	RtlUserThreadStart [0x00007FF9CA15A94C+44]

Error processing row: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7A3A99632+30946]
	(No symbol) [0x00007FF7A3A4E3C9]
	(No symbol) 

In [17]:
#Create a new dataframe for each pdf file.

downloads_dir = "pdf downloads"

dataframes = {}

pdf_files = [f for f in os.listdir(downloads_dir) if f.endswith('.pdf')]

for pdf_file in pdf_files:
    date_part = pdf_file.split('_')[-1].replace('.pdf', '')
    pdf_path = os.path.join(downloads_dir, pdf_file)

    df = tabula.read_pdf(pdf_path, stream=True, pages=1)[0]

    dataframes[f"df_{date_part}"] = df

In [20]:
import pandas as pd

def process_rows(df, rate_value):
    supplier_index = df[df.iloc[:, 0].str.contains("SUPPLIER", na=False)].index[0]
    df_new = df.iloc[supplier_index + 1:]

    df_new = df_new.reset_index(drop=True)

    wesm_index = df_new[df_new.iloc[:, 0].str.contains("PHILIPPINES", na=False, case=False, regex=False)].index
    if not wesm_index.empty:
        wesm_index = wesm_index[0]
        df_new = df_new.iloc[:wesm_index + 1]

    # Reset index for the cleaned DataFrame
    df_new = df_new.reset_index(drop=True)

    # Extract the row containing "TOTAL" separately
    total_row = df[df.iloc[:, 0].str.contains("TOTAL", na=False, case=True, regex=False)]
    if not total_row.empty:
        total_row = total_row.reset_index(drop=True)
    else:
        total_row = None

    # Append the TOTAL row if it exists
    if total_row is not None:
        df_new = pd.concat([df_new, total_row], ignore_index=True)

    df_new = df_new[df_new.iloc[:, 0].str.contains(r'\d|PHILIPPINES|TOTAL', na=False, case=True)]

    # Reset index for the cleaned DataFrame
    df_new = df_new.reset_index(drop=True)

    # Identify columns based on their content
    col_with_comma = None
    col_with_dot = None

    # Find the first column with a comma
    for i in range(1, len(df_new.columns)):
        if df_new.iloc[:, i].dtype == object:
            if col_with_comma is None and df_new.iloc[:, i].str.contains(',', na=False).any():
                col_with_comma = i

    # Find the last column with a dot
    for i in range(len(df_new.columns) - 1, -1, -1):
        if df_new.iloc[:, i].dtype == object:
            if df_new.iloc[:, i].str.contains(r'\.', na=False).any():
                col_with_dot = i
                break

    columns_to_include = [0]  # Always include the first column

    if col_with_comma is not None:
        columns_to_include.append(col_with_comma)

    if col_with_dot is not None and col_with_dot not in columns_to_include:
        columns_to_include.append(col_with_dot)

    if len(columns_to_include) == 1:
        # If only one column to include, default to the first two columns
        columns_to_include = [0, 1]

    df_new = df_new.iloc[:, columns_to_include]
    df_new.columns = ['Power Suppliers', 'kWh', 'Average Generation Cost']

    # Remove numbers in Power Suppliers
    df_new['Power Suppliers'] = df_new['Power Suppliers'].str.replace(r'^\d+\.\s*', '', regex=True)

    df_new['Generation Charge'] = rate_value

    return df_new


In [21]:
def add_date(df,key):
    df = process_rows(dataframes[key])

    end_part = key.split("_")[1]

    month_map = {
        '01': 'Jan', '02': 'Feb', '03': 'Mar', '04': 'Apr',
        '05': 'May', '06': 'Jun', '07': 'Jul', '08': 'Aug',
        '09': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'
    }
    month, year = end_part.split('-')
    formatted_date = f"{month_map[month]}-{year}"

    # Add the new column to the dataframe
    df['Date'] = formatted_date

    return(df)

In [25]:
import pandas as pd

# Create sample DataFrames (Replace these with your actual DataFrames)
df1 = pd.DataFrame({
    'Column1': ['SUPPLIER A', 'Item 1', 'Item 2'],
    'Column2': ['Data1', 'Data2', 'Data3']
})
df2 = pd.DataFrame({
    'Column1': ['SUPPLIER B', 'Item 3', 'Item 4'],
    'Column2': ['Data4', 'Data5', 'Data6']
})

# List of DataFrames to process
list_of_dataframes = [df1, df2]

# Define a rate value (Replace this with your actual rate value)
rate_value = 5.0

# Function to process rows
def process_rows(df, rate_value):
    supplier_index = df[df.iloc[:, 0].str.contains("SUPPLIER", na=False)].index[0]
    df_new = df.iloc[supplier_index + 1:]

    df_new = df_new.reset_index(drop=True)

    wesm_index = df_new[df_new.iloc[:, 0].str.contains("PHILIPPINES", na=False, case=False, regex=False)].index
    if not wesm_index.empty:
        wesm_index = wesm_index[0]
        df_new = df_new.iloc[:wesm_index + 1]

    df_new = df_new.reset_index(drop=True)

    total_row = df[df.iloc[:, 0].str.contains("TOTAL", na=False, case=True, regex=False)]
    if not total_row.empty:
        total_row = total_row.reset_index(drop=True)
    else:
        total_row = None

    if total_row is not None:
        df_new = pd.concat([df_new, total_row], ignore_index=True)

    df_new = df_new[df_new.iloc[:, 0].str.contains(r'\d|PHILIPPINES|TOTAL', na=False, case=True)]
    df_new = df_new.reset_index(drop=True)

    col_with_comma = None
    col_with_dot = None

    for i in range(1, len(df_new.columns)):
        if df_new.iloc[:, i].dtype == object:
            if col_with_comma is None and df_new.iloc[:, i].str.contains(',', na=False).any():
                col_with_comma = i

    for i in range(len(df_new.columns) - 1, -1, -1):
        if df_new.iloc[:, i].dtype == object:
            if df_new.iloc[:, i].str.contains(r'\.', na=False).any():
                col_with_dot = i
                break

    columns_to_include = [0]

    if col_with_comma is not None:
        columns_to_include.append(col_with_comma)

    if col_with_dot is not None and col_with_dot not in columns_to_include:
        columns_to_include.append(col_with_dot)

    if len(columns_to_include) == 1:
        columns_to_include = [0, 1]

    df_new = df_new.iloc[:, columns_to_include]
    df_new.columns = ['Power Suppliers', 'kWh', 'Average Generation Cost']

    df_new['Power Suppliers'] = df_new['Power Suppliers'].str.replace(r'^\d+\.\s*', '', regex=True)
    df_new['Generation Charge'] = rate_value

    return df_new

# List to store processed DataFrames
processed_dfs = []

# Process each DataFrame and collect the results
for df in list_of_dataframes:
    processed_df = process_rows(df, rate_value)
    if processed_df is not None and not processed_df.empty:
        processed_dfs.append(processed_df)
    else:
        print("Processed DataFrame is empty or None. Skipping...")

# Concatenate all the processed DataFrames into one big DataFrame
if processed_dfs:
    big_df = pd.concat(processed_dfs, ignore_index=True)
    print(big_df)
else:
    print("No objects to concatenate. The list of processed DataFrames is empty.")


ValueError: Length mismatch: Expected axis has 2 elements, new values have 3 elements